# Assignment 1

Using text http://www.gutenberg.org/files/2600/2600-0.txt
1. Make text lowercase and remove all punctuation except spaces and dots.
2. Tokenize text by BPE with vocab_size = 100
3. Train 3-gram language model with laplace smoothing $\delta=1$
4. Using beam search with k=10 generate sequences of length=10 conditioned on provided inputs. Treat dots as terminal tokens.
5. Calculate perplexity of the language model for the first sentence.

In [439]:
text = open('peace.txt', 'r', encoding="UTF-8").read()[2:]
len(text)

3227579

In [440]:
import string
import re

def preprocess_text(text):
    text = text.lower()
    punct = "“”‘’—" + string.punctuation.replace('.', '')
    for char in punct:
        text = text.replace(char, " ")
    return re.sub('\s+', ' ', text)

text = preprocess_text(text)
assert len(text) == 3141169

In [441]:
text = text.split('.')
text = [x.strip() for x in text]

In [442]:
from progressbar import Percentage, Bar, ETA, FileTransferSpeed
from IPython.display import clear_output

widgets = [Percentage(), ' ', Bar(marker='0',left='[',right=']'), ' ', ETA(), ' ', FileTransferSpeed()]

In [443]:
from collections import Counter
import nltk
from sklearn.base import TransformerMixin
from copy import deepcopy


class BPE(TransformerMixin):
    def __init__(self, vocab_size=100):
        super(BPE, self).__init__()
        self.vocab_size = vocab_size
        # index to token
        self.itos = []
        # token to index
        self.stoi = {}
        
    @staticmethod
    def update_encoding(text, new_token, new_id):
        new_text, i = [], 0
        
        while i < len(text):
            if i == len(text) - 1:
                new_text.append(text[i])
            elif (text[i], text[i + 1]) == new_token:
                new_text.append(new_id)
                i += 1
            else:
                new_text.append(text[i])
            i += 1
        return new_text
        
    def fit(self, text):
        """
        fit itos and stoi
        text: list of strings 
        """
        
        # TODO
        # tokenize text by symbols and fill in self.itos and self.stoi
        text_ = deepcopy(" ".join(text))
        self.itos = list(set(list(text_)))
        self.stoi = {token: i for i, token in enumerate(self.itos)}
        text_ = [self.stoi[char] for char in text_]
        
        pbar = ProgressBar(widgets=["fitting: "] + widgets, maxval=self.vocab_size)
        pbar.start()
        
        while len(self.itos) < self.vocab_size:
            new_token = Counter([(text_[i], text_[i+1]) for i in range(len(text_) - 1)]).most_common(1)[0]
            new_id = len(self.itos)
            
            self.itos.append(new_token)
            self.stoi[new_token] = new_id
            
            # find occurences of the new_token in the text and replace them with new_id
            text_ = self.update_encoding(text_, new_token, new_id)
            pbar.update(len(self.itos))
        pbar.finish()
        return self
    
    def transform(self, text):
        """
        convert text to a sequence of token ids
        text: list of strings
        """
        clear_output()
        text_ =  deepcopy(text)
        pbar = ProgressBar(widgets=["transforming: "] + widgets, maxval=len(text))
        pbar.start()
        for i, sent in enumerate(text_):
            token_sent = [self.stoi[char] for char in sent]
            for token_id, token in enumerate(self.itos):
                text_[i] = self.update_encoding(token_sent, token, token_id)
            pbar.update(i)
        pbar.finish()
        return text_
    
    def decode_token(self, tok):
        """
        tok: int or tuple
        """
        result = ""
        
        def recursive_search(token):
            if type(token) == str:
                nonlocal result
                result += token
            elif type(token) == int:
                recursive_search(self.itos[token])
            else:
                for el in token:
                    recursive_search(token)
                    
        recursive_search(tok)
        return result
            
    def decode(self, text):
        """
        convert token ids into text
        """
        return ''.join(map(self.decode_token, text))
    
vocab_size = 100
bpe = BPE(vocab_size)
tokenized_text = bpe.fit_transform(text)

transforming: 100% [00000000000000000000000000000000] Time: 0:01:16 404.37  B/s


In [444]:
assert bpe.decode(tokenized_text[0]) == text[0]

In [445]:
import numpy as np
        
    
start_token = vocab_size
end_token = vocab_size + 1
        
    
class LM:
    def __init__(self, vocab_size, delta=1):
        self.delta = delta
        self.vocab_size = vocab_size + 2
        self.proba = {}
        
    def smoothen_count(self, count, tau):
        return (count + self.delta) ** (1/tau)
        
    def infer(self, a, b, tau=1):
        """
        return vector of probabilities of size self.vocab for 3-grams which start with (a,b) tokens
        a: first token id
        b: second token id
        tau: temperature
        """
        result = []
        for token in range(self.vocab_size):
            result.append(self.get_proba(a, b, token, tau))
        return np.array(result)
        
    def get_proba(self, a, b, c, tau=1):
        """
        get probability of 3-gram (a,b,c)
        a: first token id
        b: second token id
        c: third token id
        tau: temperature
        """
        all_tri_proba = []
        for token in range(self.vocab_size):
            all_tri_proba.append(self.smoothen_count(self.proba[(a, b, token)], tau))
        return self.smoothen_count(self.proba[(a, b, c)], tau) / sum(all_tri_proba)
    
    def fit(self, text):
        """
        train language model on text
        text: list of lists
        """
        
        trigrams = []
        for sent in text:
            pre_sent = [start_token] + sent + [end_token]
            for i in range(len(pre_sent) - 2):
                trigrams.append((pre_sent[i], pre_sent[i + 1], pre_sent[i + 2]))
        self.proba = Counter(trigrams)
        
        return self
    
lm = LM(vocab_size, 1).fit(tokenized_text)

In [483]:
def get_top_k_probs(probs, k):
    out = []
    sorted_probs = sorted(probs, reverse=True)
    for i in range(k):
        out += np.argwhere(probs == sorted_probs[i]).flatten().tolist()
    return out[:k]      

In [484]:
def beam_search(input_seq, lm, max_len=10, k=5, tau=1):
    """
    generate sequence from language model *lm* conditioned on input_seq
    input_seq: sequence of token ids for conditioning
    lm: language model
    max_len: max generated sequence length
    k: size of beam
    tau: temperature
    """
    probs = np.log(lm.infer(input_seq[-2], input_seq[-1], tau))
    best_probs = get_top_k_probs(probs, k)
    beam = [(input_seq + [tok], probs[tok]) for tok in best_probs]
    
    for i in range(max_len):
        candidates = []
        candidates_proba = []
        for snt, snt_proba in beam:
            if snt == end_token:
                continue
            else:    
                proba = lm.infer(snt[-2], snt[-1], tau)
                best_k = get_top_k_probs(proba, k)
                candidates += [snt + [token] for token in best_k]
                candidates_proba += [snt_proba + np.log(proba)[snt] for snt in best_k]

        idxs = get_top_k_probs(np.array(candidates_proba), k)        
        beam = [(candidates[k], candidates_proba[k]) for k in idxs]
    return beam
    

In [485]:
input1 = 'horse '
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
for pair in result:
    print(f"sent: {bpe.decode(pair[0])}; log proba {pair[1]}")    

transforming: 100% [00000000000000000000000000000000] Time: 0:00:00 479.90  B/s


sent: horse and the and; log proba -1.004834169269579
sent: horse and the the; log proba -2.151178682936257
sent: horse the and the; log proba -2.151178682936258
sent: horse and the sai; log proba -3.3330720726256624
sent: horse said the an; log proba -3.5864795087903008
sent: horse the the and; log proba -3.776752547777372
sent: horse and the she; log proba -4.022227590881958
sent: horse she and the; log proba -4.022227590881958
sent: horse and the she; log proba -4.022227590881958
sent: horse she and the; log proba -4.022227590881958


In [486]:
input1 = 'her'
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
for pair in result:
    print(f"sent: {bpe.decode(pair[0])}; log proba {pair[1]}")

transforming: 100% [00000000000000000000000000000000] Time: 0:00:00 358.46  B/s


sent: her the and th; log proba -0.8758635374400061
sent: her and the an; log proba -1.7327274685663325
sent: her the the an; log proba -2.5014367096546333
sent: her and the th; log proba -2.879072674859498
sent: her the said t; log proba -3.4574831741447314
sent: her the the th; log proba -3.6477819159477987
sent: her the so the; log proba -3.9096886346987563
sent: her and the sa; log proba -4.060937542958939
sent: her and the so; log proba -4.278740780453431
sent: her the she an; log proba -4.372485617600334


In [487]:
input1 = 'what'
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=1)
for pair in result:
    print(f"sent: {bpe.decode(pair[0])}; log proba {pair[1]}")

transforming: 100% [00000000000000000000000000000000] Time: 0:00:00   1.14 kB/s


sent: what of the the; log proba -9.64722914830439
sent: what the and th; log proba -9.702569245364419
sent: what of the and; log proba -10.078457690711874
sent: what the the th; log proba -10.097594847851182
sent: what and the th; log proba -10.26657083221423
sent: what the the an; log proba -10.454168661595848
sent: what the and an; log proba -10.603040803225321
sent: what and the an; log proba -10.623144645958895
sent: what the and to; log proba -10.756569487782928
sent: what the the to; log proba -11.15159509026969


In [488]:
input1 = 'gun '
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
for pair in result:
    print(f"sent: {bpe.decode(pair[0])}; log proba {pair[1]}")

transforming: 100% [00000000000000000000000000000000] Time: 0:00:00 260.14  B/s


sent: gun the and the; log proba -0.5170415091234943
sent: gun the the and; log proba -2.1426153739646088
sent: gun the said th; log proba -3.098687541270703
sent: gun the the the; log proba -3.288959887631287
sent: gun the so the ; log proba -3.550867513329118
sent: gun the she and; log proba -4.013664281910308
sent: gun the saing t; log proba -4.347417987372495
sent: gun the the sai; log proba -4.470853277320692
sent: gun the was and; log proba -4.642095694783078
sent: gun the the so ; log proba -5.161876199009303


In [505]:
from math import log, exp

def perplexity(snt, lm):
    """
    snt: sequence of token ids
    lm: language model
    """
    perplexity = 0
    
    snt = [start_token] + snt + [end_token]

    for char in range(len(snt) - 2):
        perplexity += log((1 / lm.infer(snt[char], snt[char + 1])[snt[char + 2]]))
    result = pow(perplexity, -1 / float(len(snt)))
    return exp(result)

perplexity(tokenized_text[0], lm)

2.663941575945858